# Pre-processing Data Waveform
Working Group Member 3 BMKG

***Authors:***
1.  Iman Fatchurochman, S.Si, M.DM
2.  Nova Heryandoko, S.Si, M.Si
3.  Ajat Sudrajat, M.Sc
4.  Bayu Pranata, M.Si
5.  Arif Nurokhim
6.  Kian Purna Sinki, SST, M.DM
7.  Melky Adi Kurniawan, S.Si
8.  Robby Wallansha, S.Tr
9.  Abdul Rosid, S.Tr, M.DM
10. Muhammad Fahmi Nugraha, S.Tr
11. Sesar Prabu Dwi Sriyanto
12. Ade Andika Saputra, S.Tr.Geof.
13. Rosi Budi Kurniawan, S.Tr.Geof.
14. Angga Wijaya, S.Tr.Geof.

In [ ]:
# Modul-modul yang diperlukan
import os
import sys
import re
from obspy import read
from obspy import read_inventory
from obspy.geodetics import locations2degrees
from obspy.geodetics import degrees2kilometers
from obspy.clients.iris.client import Client
import matplotlib.pyplot as plt

In [ ]:
# Get Parameter
dirf = sys.argv[1]

# Directory
if not os.path.isdir(dirf + '/outputs'):
    os.mkdir(dirf + '/outputs')
if not os.path.isdir(dirf + '/earthquakes'):
    os.mkdir(dirf + '/earthquakes')

# Read stasiun yang digunakan txt
file = open(dirf + '/sta_noise.txt')
sta = file.readlines()
file.close()

# Read data event txt
file = open('list_gempa_original.txt')
event = file.readlines()
file.close()

regex = re.split("T|_", dirf)
mag = f"{regex[5][0]}.{regex[5][1]}"
depth = f"{regex[4]}"

for i in range(len(sta)):
    sta[i] = sta[i].split()

for i in range(len(event)):
    event[i] = event[i].split()

event_core = []
for evt in event:
    if evt[5] == dirf:
        event_core = evt

# Create epicenter distance txt
file_e = open(dirf + f"/{dirf}_E.txt", 'w')
file_n = open(dirf + f"/{dirf}_N.txt", 'w')
file_z = open(dirf + f"/{dirf}_Z.txt", 'w')

In [ ]:
# Generate Integrasi png
file_e.write("{:<8}".format("Station") +
             "{:<13}".format("Distance(km)") +
             "{:<10}".format("Magnitude") +
             "{:<10}".format("Depth(km)") +
             "{:<11}".format("Azimuth(°)") +
             "{:>12}".format("AmpTc5s(m)") +
             "{:>14}".format("AmpTc10s(m)") +
             "{:>14}".format("AmpTc20s(m)") +
             "\n")

file_n.write("{:<8}".format("Station") +
             "{:<13}".format("Distance(km)") +
             "{:<10}".format("Magnitude") +
             "{:<10}".format("Depth(km)") +
             "{:<11}".format("Azimuth(°)") +
             "{:>12}".format("AmpTc5s(m)") +
             "{:>14}".format("AmpTc10s(m)") +
             "{:>14}".format("AmpTc20s(m)") +
             "\n")

file_z.write("{:<8}".format("Station") +
             "{:<13}".format("Distance(km)") +
             "{:<10}".format("Magnitude") +
             "{:<10}".format("Depth(km)") +
             "{:<11}".format("Azimuth(°)") +
             "{:>12}".format("AmpTc5s(m)") +
             "{:>14}".format("AmpTc10s(m)") +
             "{:>14}".format("AmpTc20s(m)") +
             "\n")

for i in sta:
    data = f"{dirf}/earthquakes/{i[0]}.mseed"
    st = read(data)

    tr_e = st[0]
    tr_n = st[1]
    tr_z = st[2]

    # Remove instrument response
    name_stat = i[0]
    inv = read_inventory("/home/dika/PUSAT/WG3/inventory/" + name_stat + ".xml")
    pre_filter = [0.001, 0.005, 40.0, 45.0]

    tr_e_cor = tr_e.copy().detrend(type='demean').taper(max_percentage=0.05, type='cosine')
    tr_n_cor = tr_n.copy().detrend(type='demean').taper(max_percentage=0.05, type='cosine')
    tr_z_cor = tr_z.copy().detrend(type='demean').taper(max_percentage=0.05, type='cosine')

    try:
        tr_e_cor.remove_response(inventory=inv, output='ACC', pre_filt=pre_filter, plot=True, water_level=60)
        tr_n_cor.remove_response(inventory=inv, output='ACC', pre_filt=pre_filter, plot=True, water_level=60)
        tr_z_cor.remove_response(inventory=inv, output='ACC', pre_filt=pre_filter, plot=True, water_level=60)
    except:
        print(f"ValueError on dataless {name_stat}, no matching response information found")
        print("\n")
        continue

    # Calculate epicenter distance
    staLat = inv[0][0].latitude
    staLon = inv[0][0].longitude
    dist_degree = locations2degrees(float(event_core[2]), float(event_core[1]), float(staLat), float(staLon))
    dist_km = round(degrees2kilometers(dist_degree), 2)

    # Calculate azimuth
    az = Client().distaz(stalat=float(staLat),
                         stalon=float(staLon),
                         evtlat=float(event_core[2]),
                         evtlon=float(event_core[1]))

    # Save img raw acceleration
    fig_e = tr_e_cor.plot(show=False, handle=True)
    plt.legend(['Raw Acceleration (m/s^2)'], loc='upper right')
    fig_e.savefig(f"{dirf}/outputs/{name_stat}_E.png")

    fig_n = tr_n_cor.plot(show=False, handle=True)
    plt.legend(['Raw Acceleration (m/s^2)'], loc='upper right')
    fig_n.savefig(f"{dirf}/outputs/{name_stat}_N.png")

    fig_z = tr_z_cor.plot(show=False, handle=True)
    plt.legend(['Raw Acceleration (m/s^2)'], loc='upper right')
    fig_z.savefig(f"{dirf}/outputs/{name_stat}_Z.png")

    # Filter Butterworth Highpas 3rd order
    freq1 = 0.2
    freq2 = 0.1
    freq3 = 0.05
    
    # Integrasi Velocity
    vel_e_1 = tr_e_cor.copy().integrate(method='cumtrapz')
    vel_e_2 = tr_e_cor.copy().integrate(method='cumtrapz')
    vel_e_3 = tr_e_cor.copy().integrate(method='cumtrapz')

    vel_n_1 = tr_n_cor.copy().integrate(method='cumtrapz')
    vel_n_2 = tr_n_cor.copy().integrate(method='cumtrapz')
    vel_n_3 = tr_n_cor.copy().integrate(method='cumtrapz')

    vel_z_1 = tr_z_cor.copy().integrate(method='cumtrapz')
    vel_z_2 = tr_z_cor.copy().integrate(method='cumtrapz')
    vel_z_3 = tr_z_cor.copy().integrate(method='cumtrapz')

    # Integrasi Displacement
    disp_e_1 = vel_e_1.copy().integrate(method='cumtrapz')
    disp_e_2 = vel_e_2.copy().integrate(method='cumtrapz')
    disp_e_3 = vel_e_3.copy().integrate(method='cumtrapz')

    disp_n_1 = vel_n_1.copy().integrate(method='cumtrapz')
    disp_n_2 = vel_n_2.copy().integrate(method='cumtrapz')
    disp_n_3 = vel_n_3.copy().integrate(method='cumtrapz')

    disp_z_1 = vel_z_1.copy().integrate(method='cumtrapz')
    disp_z_2 = vel_z_2.copy().integrate(method='cumtrapz')
    disp_z_3 = vel_z_3.copy().integrate(method='cumtrapz')

    # Filter
    tr_e_cor_1 = disp_e_1.copy()
    tr_e_cor_2 = disp_e_2.copy()
    tr_e_cor_3 = disp_e_3.copy()
    tr_e_cor_1.filter("highpass", freq=freq1, corners=3)
    tr_e_cor_2.filter("highpass", freq=freq2, corners=3)
    tr_e_cor_3.filter("highpass", freq=freq3, corners=3)

    tr_n_cor_1 = disp_n_1.copy()
    tr_n_cor_2 = disp_n_2.copy()
    tr_n_cor_3 = disp_n_3.copy()
    tr_n_cor_1.filter("highpass", freq=freq1, corners=3)
    tr_n_cor_2.filter("highpass", freq=freq2, corners=3)
    tr_n_cor_3.filter("highpass", freq=freq3, corners=3)

    tr_z_cor_1 = disp_z_1.copy()
    tr_z_cor_2 = disp_z_2.copy()
    tr_z_cor_3 = disp_z_3.copy()
    tr_z_cor_1.filter("highpass", freq=freq1, corners=3)
    tr_z_cor_2.filter("highpass", freq=freq2, corners=3)
    tr_z_cor_3.filter("highpass", freq=freq3, corners=3)

    file_e.write("{:<8}".format(f"{name_stat}") +
                 "{:>12}".format(f"{dist_km:.2f}") +
                 "{:^10}".format(f"{mag}") +
                 "{:^10}".format(f"{depth}") +
                 "{:>11}".format(f"{az['azimuth']:.2f} ") +
                 "{:>14}".format(f"{abs(tr_e_cor_1.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_e_cor_2.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_e_cor_3.data).max():.10f}\n"))

    file_n.write("{:<8}".format(f"{name_stat}") +
                 "{:>12}".format(f"{dist_km:.2f}") +
                 "{:^10}".format(f"{mag}") +
                 "{:^10}".format(f"{depth}") +
                 "{:>11}".format(f"{az['azimuth']:.2f} ") +
                 "{:>14}".format(f"{abs(tr_n_cor_1.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_n_cor_2.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_n_cor_3.data).max():.10f}\n"))

    file_z.write("{:<8}".format(f"{name_stat}") +
                 "{:>12}".format(f"{dist_km:.2f}") +
                 "{:^10}".format(f"{mag}") +
                 "{:^10}".format(f"{depth}") +
                 "{:>11}".format(f"{az['azimuth']:.2f} ") +
                 "{:>14}".format(f"{abs(tr_z_cor_1.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_z_cor_2.data).max():.10f} ") +
                 "{:>14}".format(f"{abs(tr_z_cor_3.data).max():.10f}\n"))

    # Save img displacement
    name_e_1 = f"{dirf}/outputs/{name_stat}_E_{freq1}.png"
    name_e_2 = f"{dirf}/outputs/{name_stat}_E_{freq2}.png"
    name_e_3 = f"{dirf}/outputs/{name_stat}_E_{freq3}.png"

    name_n_1 = f"{dirf}/outputs/{name_stat}_N_{freq1}.png"
    name_n_2 = f"{dirf}/outputs/{name_stat}_N_{freq2}.png"
    name_n_3 = f"{dirf}/outputs/{name_stat}_N_{freq3}.png"

    name_z_1 = f"{dirf}/outputs/{name_stat}_Z_{freq1}.png"
    name_z_2 = f"{dirf}/outputs/{name_stat}_Z_{freq2}.png"
    name_z_3 = f"{dirf}/outputs/{name_stat}_Z_{freq3}.png"

    fig = tr_e_cor_1.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=5s)'], loc='lower left')
    fig.savefig(name_e_1)
    plt.close()

    fig = tr_e_cor_2.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=10s)'], loc='lower left')
    fig.savefig(name_e_2)
    plt.close()

    fig = tr_e_cor_3.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=20s)'], loc='lower left')
    fig.savefig(name_e_3)
    plt.close()

    fig = tr_n_cor_1.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=5s)'], loc='lower left')
    fig.savefig(name_n_1)
    plt.close()

    fig = tr_n_cor_2.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=10s)'], loc='lower left')
    fig.savefig(name_n_2)
    plt.close()

    fig = tr_n_cor_3.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=20s)'], loc='lower left')
    fig.savefig(name_n_3)
    plt.close()

    fig = tr_z_cor_1.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=5s)'], loc='lower left')
    fig.savefig(name_z_1)
    plt.close()

    fig = tr_z_cor_2.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=10s)'], loc='lower left')
    fig.savefig(name_z_2)
    plt.close()

    fig = tr_z_cor_3.plot(show=False, handle=True)
    plt.legend(['Displacement (Tc=20s)'], loc='lower left')
    fig.savefig(name_z_3)
    plt.close()

    plt.rcParams.update({'figure.max_open_warning': 0})

file_e.close()
file_n.close()
file_z.close()